In [ ]:
from google.colab import output

output.clear()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
output.clear()

In [ ]:
!apt install chromium-chromedriver
output.clear()

In [ ]:
# For Selenium n Google Colab
!pip install selenium
!apt-get update 
!pip install webdriver-manager
output.clear()

In [ ]:
from webdriver_manager.firefox import DriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
output.clear()

In [ ]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
output.clear()

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
output.clear()

In [ ]:
import pandas as pd
import time
import os
import io
import numpy as np
import requests
from datetime import datetime 
import csv
import re
from bs4 import BeautifulSoup
output.clear()

In [ ]:
url = 'https://www.naukri.com/{name}-jobs{num}?k={name}'
def get_url(x,pagenum):
    y = x.split()
    return url.format(name = "-".join(y),num=pagenum)
output.clear()

The files below should be in the same folder as the code for importing the semantics and education file.

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Semantic Titles.xlsx',sheet_name='Sheet1')
df.fillna('missing',inplace=True)
df['scrapenum'] = 0
for i in range(0,len(df)):
    #print('\n')
    if((i>0) and (df['scrapenum'][i-1]==0) and (cnt>0)):
        df['scrapenum'][i-1]=cnt-1
    cnt = 0 
    for j in df.columns:
        #print(df[j][i])
        if(df[j][i]=='missing'):
            df['scrapenum'][i] = cnt-1
            break
        cnt = cnt+1
output.clear()

In [ ]:
certis = {}
for i in df['Job Title']:
  certis[i] = ""

In [ ]:
cert = pd.read_csv('/content/drive/MyDrive/data/certificates.csv') 
for i in range(0,len(cert)):
  certis[cert['Job'][i].rstrip().lstrip()] = cert['Certificates'][i]

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver1 = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
output.clear()

In [ ]:
def repl(x):
  return x.replace("\t","").replace("\n","").rstrip().lstrip().replace("  "," ")

In [ ]:
dfs = pd.DataFrame()
fincnt = 1
for ind in range(0,len(df)):
    if(df['scrapenum'][ind]<=1):
        counter = 50
    else:
        counter = 50//df['scrapenum'][ind]
    if(counter==1):
      counter+=1
    experience,name,skills,desc,location,salary,education,links,semantics,certifications=[],[],[],[],[],[],[],[],[],[]
    if(df['Job Title'][0]==df['Semantic Words'][0]):
        applist = df.drop(['Semantic Words','scrapenum'],axis=1).columns
    else:
        applist = df.drop('scrapenum',axis=1).columns
    fincnt+=1
    if(counter<=20):
      print("----------------# ",df['Job Title'][ind],"Page (1)------------")
    for col in applist:
        if(df[col][ind]=='missing'):
            break
        else:
            if(counter<=20):  
                driver.get(get_url(df[col][ind],""))
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)
                c = []
                count = 0
                for i in driver.find_elements_by_xpath("//div['list']//article['mb-8']"):
                    if(count==counter):
                        break
                    count+=1
                    try:
                      t = i.find_element_by_css_selector('a')
                      links.append(t.get_attribute('href'))
                    except:
                      links.append('None')
                      desc.append('None')
                      education.append('None')
                      #print('link error')
                    if(links[len(links)-1]!='None'):
                      time.sleep(3)
                      driver1.get(t.get_attribute('href'))
                      desctext,eductext = "",""
                      try:
                        desctext = driver1.find_element_by_class_name("dang-inner-html").text
                      except NoSuchElementException:
                        desctext = "None"
                      try:
                        eductext = driver1.find_element_by_class_name("education").text
                      except NoSuchElementException: 
                        eductext =  "None"
                      certitext = ""
                      for cer in certis[df['Job Title'][ind]].split(","):
                          if(re.search(cer.lstrip().rstrip(),desctext)):
                            certitext = cer+ " "+certitext 
                      if(certitext==''):
                        certitext= 'None'
                      certifications.append(certitext)
                      desc.append(desctext)
                      education.append(eductext)
                            
                        
                    try:
                        k = i.find_elements_by_css_selector('span.ellipsis')
                        cntr = 0
                        for ki in k:
                            if(cntr==0):
                                experience.append(ki.text)
                            elif(cntr==1):
                                salary.append(ki.text)
                            else:
                                location.append(ki.text)
                            cntr+=1
                            #print(ki.text)
                    except NoSuchElementException:
                        experience.append('None')
                        salary.append('None')
                        location.append('None')
                        #print('error in span.ellipsis')
                    try:
                        semantics.append(df[col][ind])
                        name.append(i.find_element_by_css_selector('div.mt-7').find_element_by_css_selector('a').text)
                    except NoSuchElementException:
                        name.append('None')
                    try:
                        c.append(i.find_element_by_css_selector('ul.has-description').text)
                    except NoSuchElementException:
                        c.append('None')
                for i in c:
                    s = i.replace(" ",'-')
                    s = s.replace("\n"," ")
                    skills.append(s)
            else:
                f = True
                cnt1 = 0
                while(f):
                    print("----------------# ",df['Job Title'][ind],"Page (",cnt1+1,")------------")
                    if(cnt1==0):
                        driver.get(get_url(df[col][ind],""))
                    else:
                        driver.get(get_url(df[col][ind],"-"+str(cnt1+1)))
                    cnt1=cnt1+1
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(3)
                    c = []
                    for i in driver.find_elements_by_xpath("//div['list']//article['mb-8']"):
                        try:
                            t = i.find_element_by_css_selector('a')
                            links.append(t.get_attribute('href'))
                        except:
                            links.append('None')
                            desc.append('None')
                            education.append('None')
                        if(links[len(links)-1]!='None'):
                          driver1.get(t.get_attribute('href'))
                          time.sleep(3)
                          desctext,eductext = "",""
                          try:
                            desctext = driver1.find_element_by_class_name("dang-inner-html").text
                          except NoSuchElementException:
                            desctext = "None"
                          try:
                            eductext = driver1.find_element_by_class_name("education").text
                          except NoSuchElementException: 
                            eductext =  "None"
                          certitext = ""
                          for cer in certis[df['Job Title'][ind]].split(","):
                              if(re.search(cer.lstrip().rstrip(),desctext)):
                                certitext = cer+ " "+certitext 
                          if(certitext==''):
                            certitext= 'None'
                          certifications.append(certitext)
                          desc.append(desctext)
                          education.append(eductext)
                          
                        try:
                            k = i.find_elements_by_css_selector('span.ellipsis')
                            cntr = 0
                            for ki in k:
                                if(cntr==0):
                                    experience.append(ki.text)
                                elif(cntr==1):
                                    salary.append(ki.text)
                                else:
                                    location.append(ki.text)
                                cntr+=1
                        except NoSuchElementException:
                            experience.append('None')
                            salary.append('None')
                            location.append('None')

                        try:
                            semantics.append(df[col][ind])
                            name.append(i.find_element_by_css_selector('div.mt-7').find_element_by_css_selector('a').text)
                        except NoSuchElementException:
                            name.append('None')
                        try:
                            c.append(i.find_element_by_css_selector('ul.has-description').text)
                        except NoSuchElementException:
                            c.append('None')
      
                    for i in c: #loop for extracting the skills without special characters
                        s = i.replace(" ",'-')
                        s = s.replace("\n"," ")
                        skills.append(s)
                    if(cnt1==3):
                        f = False
    classjob = []
    for i in range(0,len(name)):
      classjob.append(df['Job Title'][ind])
    response = []
    for d in desc:
      f = True
      for res in ['responsibility','funcion','activities','activity','responsibilities','responsible']:
        if(re.search(res,d)):
          response.append(d[re.search(res,d.lower()).span()[0]:len(d)])
          f =False
          break
      if(f):
        response.append('None')
    if((len(semantics)==len(skills)) and (len(semantics)==len(name)) and
       (len(semantics)==len(desc)) and (len(semantics)==len(location)) and 
       (len(semantics)==len(response)) and (len(semantics)==len(education))
       and (len(semantics)==len(salary)) and (len(semantics)==len(experience)) and len(semantics)==len(links)):
      dftemp = pd.DataFrame({"Main Job Title":semantics,
                            "Searched Job Title":classjob,
                           "Company Name":name,
                           "skills":skills,
                           "description":desc,
                           "location":location,
                           "responsibilty/activity":response,
                           "education":education,
                           "Experience":experience,
                           "Certifications": certifications,
                           "salary":salary,
                           "links":links})
      dfs = pd.concat([dfs,dftemp])
      #dftemp.to_csv('/content/drive/MyDrive/data/naukri/{}.csv'.format(df['Job Title'][ind]))
dfs.reset_index(drop=True)
dfs['skills'] = dfs['skills'].map(repl)
dfs['description'] = dfs['description'].map(repl) 
dfs.to_csv('/content/drive/MyDrive/data/naukri/{}.csv'.format('naukri'))